# Start a Simulated SNMP Agent
This example demonstrates working with complex data types with variable length fields using composable functions and the low-level API.

In [1]:
import subprocess

try:
    if process:
        pass
except:
    process = subprocess.Popen(
        [
            'snmpsimd.py',
            '--agent-udpv4-endpoint=127.0.0.1:1161',
            '--agent-udpv6-endpoint=[::1]:1161'
        ],
        stdout=subprocess.DEVNULL,
        stderr=subprocess.DEVNULL,
        shell=False
    )

# Construct a MIB-like Definition of the SNMP Objects to Collect

In [2]:
import numpy as np
import pandas as pd

from snmp_fetch import ObjectType, object_type, pipeline_hook

df = pd.DataFrame(
    data=[
        ['localhost:1161', 'recorded/linux-full-walk'],
    ],
    columns=['ip_address', 'community_string']
)

@object_type(oid='.1.3.6.1.2.1.4.24.7.1.9')
class StreamTest(ObjectType):
    index=np.dtype([('_buffer', (np.uint64, 128))])
    dtype=np.dtype([('value', np.uint64)])
    
    @pipeline_hook('before_pivot')
    def process_buffer(df):
        df[['dest_type', '_buffer']] = df['_buffer'].inet.buffer[0, 1:]
        df['dest_type'].astype(pd.UInt64Dtype())
        
        # assume dest_zone will be in the db index and cannot be NULL
        df[['dest_inet_address', '_buffer']] = df['_buffer'].inet.buffer.chunk()
        df[['dest_inet_address', 'dest_zone']] = df['dest_inet_address'].inet.to_inet_address(default_zone=-1)
        df['dest_zone'] = df['dest_zone'].astype(pd.Int64Dtype())
        
        df[['dest_prefix_len', '_buffer']] = df['_buffer'].inet.buffer[0, 1:]  # temp column to make cidr address

        df[['route_policy', '_buffer']] = df['_buffer'].inet.buffer.chunk()
        df['route_policy'] = df['route_policy'].inet.to_object_identifier()
        
        df[['next_hop_type', '_buffer']] = df['_buffer'].inet.buffer[0, 1:]
        df['next_hop_type'] = df['next_hop_type'].astype(pd.UInt64Dtype())
        
        df[['next_hop_inet_address', '_buffer']] = df['_buffer'].inet.buffer.chunk()
        df[['next_hop_inet_address', 'next_hop_zone']] = df['next_hop_inet_address'].inet.to_inet_address()
        df['next_hop_zone'] = df['next_hop_zone'].astype(pd.UInt64Dtype())
        
        df['dest_inet_address'] = df[['dest_inet_address', 'dest_prefix_len']].inet.to_cidr_address()
        
        df = df.drop(columns=['_buffer', 'dest_prefix_len'])
        return df

In [3]:
from snmp_fetch import PduType
from snmp_fetch.api import fetch

results, errors = fetch(
    PduType.BULKGET,
    [(i, str(h), c) for i, [h, c] in enumerate(df.reset_index()[['ip_address', 'community_string']].values)],
    StreamTest.null_var_binds('2')  # IPv6 only
)

StreamTest.to_pandas(results)

,#timestamp,value,dest_type,dest_inet_address,dest_zone,route_policy,next_hop_type,next_hop_inet_address,next_hop_zone
0,2019-11-18 18:37:30+00:00,2,2,fe80::/64,-1,.5,2,::,NaN
1,2019-11-18 18:37:30+00:00,2,2,fe80::212:79ff:fe62:f940/128,-1,.4,2,::,NaN
2,2019-11-18 18:37:30+00:00,2,2,ff00::/8,-1,.6,2,::,NaN


# Stop the Simulated SNMP Agent

In [4]:
process.kill()
process.communicate()
del process